In [5]:
import os
import fitz
import re

from ollama import Client
import faiss
import pandas as pd
import numpy as np
import Stemmer
from tqdm import tqdm
import gradio as gr

from llama_index.core import Document
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.core.retrievers import BaseRetriever, QueryFusionRetriever
from llama_index.core.schema import TextNode, NodeWithScore
from llama_index.retrievers.bm25 import BM25Retriever

# Connect to Ollama Server

In [6]:
client = Client(
  host='http://localhost:11434',
)

# Ingestion

In [7]:
import json
# Path to the dataset folder
DATASET_PATH = '/home/jeryl4913/lto_rag_reviewer/notebooks/extracted_text.json'

def get_text_and_metadata(input_path):
    """Load text and metadata from a file and perform chunking."""
    with open(input_path, "r", encoding="utf-8") as f:
        extracted_data = json.load(f)

    texts = []
    metadata = []

    for entry in extracted_data:
        text = entry["text"]
        source_metadata = {
            "source": entry["source"],
            "folder": entry["folder"],
            "file_name": entry["file_name"],
            "page": entry["page"],
            "title": entry["title"],
            "url": entry["url"]
        }
        texts.append(text)
        metadata.append(source_metadata)

    return texts, metadata

In [8]:
docs, metadatas = get_text_and_metadata(DATASET_PATH)

In [9]:
documents = [Document(text=docs[t], metadata=metadatas[t]) for t in range(len(docs))]
splitter = TokenTextSplitter(
    chunk_size=512,
    chunk_overlap=20,
    separator=" ",
)
nodes = splitter.get_nodes_from_documents(documents)

# Embedding and Retrieval

## Dense via FAISS

In [10]:
def generate_embeddings(nodes, client, model):
    # Generate embeddings for documents using Ollama
    for doc in tqdm(nodes):
        response = client.embeddings(prompt=doc.text, model=model)
        doc.embedding = response["embedding"]
    return nodes

In [11]:
class FaissIndexer:
    """
    Faiss-based indexer for efficient similarity search using inner-product (cosine) similarity.

    This class handles the creation and management of a FAISS index from node embeddings.
    
    :ivar faiss_index: The FAISS index for storing and querying embeddings.
    :vartype faiss_index: faiss.IndexFlatIP
    :ivar embedding_dim: Dimensionality of the embeddings.
    :vartype embedding_dim: int
    """

    def __init__(self):
        """
        Initialize the FaissIndexer class.

        :ivar faiss_index: The FAISS index, initialized as None.
        :ivar embedding_dim: The dimension of embeddings, initialized as None.
        """
        self.faiss_index = None
        self.embedding_dim = None

    def normalize_embeddings(self, embeddings):
        """
        Normalize embeddings to have unit L2 norm.

        :param embeddings: Array of embeddings to normalize.
        :type embeddings: np.ndarray
        :return: Normalized embeddings.
        :rtype: np.ndarray
        """
        return embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

    def build_index(self, nodes):
        """
        Build the FAISS index from a list of nodes containing embeddings.

        :param nodes: List of nodes, where each node contains an `embedding` attribute.
        :type nodes: list
        :raises ValueError: If the nodes list is empty or embeddings are inconsistent.
        """
        if not nodes:
            raise ValueError("Nodes list cannot be empty.")
        
        embeddings = np.array([np.array(node.embedding) for node in nodes])
        normalized_embeddings = self.normalize_embeddings(embeddings)

        self.embedding_dim = normalized_embeddings[0].shape[0]
        self.faiss_index = faiss.IndexFlatIP(self.embedding_dim)  # Inner-product similarity
        self.faiss_index.add(normalized_embeddings)

    def get_index(self):
        """
        Get the FAISS index instance.

        :return: The FAISS index used for similarity search.
        :rtype: faiss.IndexFlatIP
        :raises ValueError: If the index has not been built.
        """
        if self.faiss_index is None:
            raise ValueError("Index has not been built yet. Call 'build_index' first.")
        return self.faiss_index

In [12]:
class FAISSVectorStoreRetriever(BaseRetriever):
    def __init__(self, faiss_index, documents):
        """
        Initialize the FAISS retriever.
        :param faiss_index: The FAISS index containing precomputed embeddings.
        :param documents: List of document chunks.
        :param embeddings: Precomputed embeddings corresponding to the document chunks.
        """
        self.faiss_index = faiss_index
        self.documents = documents

    def _retrieve(self, query_embedding, top_k=5):
        """
        Retrieve the top-k nearest neighbors using the FAISS index.
        :param query_embedding: The embedding of the query.
        :param top_k: Number of top results to retrieve.
        """

        norm_query_embedding = np.array([query_embedding])
        norm_query_embedding /= np.linalg.norm(norm_query_embedding, axis=1, keepdims=True)

        distances, indices = self.faiss_index.search(norm_query_embedding, top_k)
        retrieved_docs = [
            NodeWithScore(node=self.documents[idx], score=1 - dist)
            for idx, dist in zip(indices[0], distances[0])
            if idx != -1
        ]
        return retrieved_docs

In [13]:
# embedding
nodes_embed = generate_embeddings(nodes, client, "mxbai-embed-large")

100%|██████████| 12302/12302 [07:46<00:00, 26.36it/s]


In [14]:
#indexing
index = FaissIndexer()
index.build_index(nodes_embed)
faiss_index = index.get_index()

faiss_retriever = FAISSVectorStoreRetriever(faiss_index=faiss_index,documents=nodes_embed)

## Sparse Embedding via BM25

In [15]:
# bm25_retriever = BM25Retriever.from_defaults(
#     nodes=nodes,
#     similarity_top_k=5,
#     stemmer=Stemmer.Stemmer("english"),
#     language="english",
# )

## Hybrid Retrieval via Reciprocal Rank

In [16]:
def hybrid_embedding(results: dict, top_k: int):
    x = QueryFusionRetriever
    ranked_results = QueryFusionRetriever._reciprocal_rerank_fusion(x, results)
    return ranked_results[:top_k]

# Post Retrieval

## Summarization

In [17]:
def summarize_each_chunk(nodes, client, query, model="llama3.3", parent=False):
    if parent:
        chunks = [doc.text for doc in nodes]
    else:
        chunks = [doc.node.text for doc in nodes]
    summaries = []
    
    for i, chunk in enumerate(chunks):
        prompt = f"""
        Summarize the following text in one concise paragraph, focusing on key points relevant to the query: "{query}".
        
        - Emphasize information directly related to the query.
        - Exclude unrelated, redundant, or speculative details.
        - Do NOT introduce new information or answer the query itself. 
        
        Text:
        {chunk}
        
        Summary:
        """
        
        response = client.generate(model=model, prompt=prompt)
        summary = response['response'].strip()
        summaries.append(summary)

    return summaries

# Generation

# Querying

## Query Transforms

In [18]:
# Few-shot examples
FEW_SHOTS = """

Example 2:
Question: What will happen when your front tire blows out?
A. The back end will sway towards the side of the blowout
B. The back end will sway away from the blowout
C. The front end will pull towards the side of the blowout
D. The front end will pull to the opposite side of the blowout
Please answer only in letters
Correct Answer: [C]

Example 3:
Question: What should you do when an ambulance comes up behind you flashing red lights and/or sounding its siren?
A. Stop as soon as you can
B. Maintain your speed, let the ambulance driver will find a way around you
C. Speed up so that you don't hold the ambulance
D. Pull over to the right and slow down or even stop if necessary
Please answer only in letters
Correct Answer: [D]
"""

def summarize_context(context: str, client, model: str) -> str:
    
    prompt_instruction = '''
    You are an expert at distilling information into concise summaries. 
    Read the following context carefully and provide a clear, concise, and accurate summary of the key points. 
    Ensure that the summary captures the essence of the content without omitting critical details. 
    
    Provide the summarized output enclosed in double quotes.
    '''

    few_shot_examples = '''
    Here is an example of how to do this

    Example 1:
    Context: High-dimensional feature vectors, derived from complex embedding models, are now critical for applications like e-commerce product search and recommendation, document retrieval, facial recognition, and retrieval-based large language models (LLMs), Pan, J. J., et al.(2024). Traditional database management systems, however, struggle with the unique characteristics of feature vectors due to the nuance of semantic similarity, expensive computational cost, large data size, and lack of indexable structure. Numerous vector database management systems (VDBMSs) have been created to tackle these issues. Typical VDBMSs consist of a query processor that handles different query types and a storage manager that manages large-scale vector indexing, storage, and compression. One key aspect of query processing is similarity scores which is important in capturing similarity relationships between feature vectors. There are various different vector indexing structures and techniques for efficient vector search. Common structures include table-based, tree-based, and graph-based.  Table-based indexes partition vectors into buckets with techniques including locality sensitive hashing (LSH), learning to hash (L2H), and quantization for compression.
    Output: "High-dimensional feature vectors are essential for applications like e-commerce search, document retrieval, facial recognition, and retrieval-based LLMs. Traditional databases struggle to manage these vectors due to their size, computational cost, and the complexity of semantic similarity. Vector database management systems (VDBMSs) address these challenges with specialized query processors and storage managers for indexing, storage, and compression. Efficient vector search relies on similarity scores and various indexing techniques, such as table-based, tree-based, and graph-based structures, with methods like locality sensitive hashing (LSH) and quantization playing a key role."
    
    Now, summarize the following context and provide the response inside double quotes:
    '''
    
    full_prompt = prompt_instruction + few_shot_examples + f"\nContext: {context}"
    
    try:
        response = client.generate(
            model=model,
            prompt=full_prompt
        )

        # Extract text within double quotes (literal summary)
        matches = re.findall(r'"([^"]*)"', response['response'].strip())

        print(f"Context: {context[:100]}...")  # Print first 100 chars for readability
        print(f"Summary: {matches[0] if matches else 'No match found'}")
        print('-' * 60)

        if matches:
            return matches[0]
        else:
            return "Summary not available."
        
    except Exception as e:
        print(f"Error during summarization: {e}")
        return "Error encountered during summarization."

def cot_qa_format(query, context, client, model):
    return f"""

    Let's break down the problem step by step:
    1. Identify the key components of the following question: {query}
    2. Decompose the retrieved context into key components and extract relevant information step by step. Here is the context: {summarize_context(query, client, model)}
    3. Provide the best possible answer based on the reasoning process.
    """

def generate_response_with_cot(summaries, query, choices, client, model="llama3.1:8b", cot_model="llama3.1:8b", para_model="llama3.1:8b"):
    """
    Generate a response using Chain of Thought (CoT) reasoning by passing intermediate
    results through successive steps to build a synthesized final answer.
    """

    # 1. Combine summaries into context block
    context = "\n".join(summaries)

    try:
        # 2. Step 1: Generate initial thought process (CoT step 1)
        cot_query = cot_qa_format(query, context, client, para_model)
        response1 = client.generate(model=cot_model, prompt=cot_query)
        reasoning1 = response1['response'].strip()

        # 3. Step 2: Refine based on first result
        cot_query2 = f"""
        We reasoned that: {reasoning1}
        Can you expand on this by addressing additional details or gaps?
        {query}
        """
        response2 = client.generate(model=cot_model, prompt=cot_query2)
        reasoning2 = response2['response'].strip()

        # 5. Aggregate all steps for the final decision
        final_query = f"""
        Based on the results:
        - Step 1: {reasoning1}
        - Step 2: {reasoning2}

        Now, synthesize these results and answer this question:
        {query + " Answer which of the choices match the synthesized results."}
        Choices: {choices}

        Please answer only in letters and put them inside brackets '[]'. If the question contains the statement 'Check all that apply', use a comma separator for multiple answers.
        Here are two examples on how to answer the question:
        {FEW_SHOTS}
        """
        
        # 6. Final model call to produce the overall response
        final_response = client.generate(model=model, prompt=final_query)
        
        return final_response['response'].strip()


    except Exception as e:
        print(f"Error processing prompt: {e}")
        return None



## Query Generation

In [19]:
docstore = {}

# Store documents using full metadata as the key
for doc in documents:
    key = tuple(doc.metadata.items())  # Convert metadata to tuple for hashable key
    docstore[key] = doc

In [20]:
def get_document_by_chunk_metadata(chunk_node):
    # Convert chunk metadata to tuple for matching
    metadata_key = tuple(chunk_node.metadata.items())

    # Retrieve document from docstore
    document = docstore.get(metadata_key)
    return document

In [21]:
def remove_duplicate_documents(doc_list):
    seen_ids = set()
    unique_docs = []

    for doc in doc_list:
        if doc.doc_id not in seen_ids:
            seen_ids.add(doc.doc_id)
            unique_docs.append(doc)

    return unique_docs

In [22]:
def gen_query(query, choices, top_k, client, mode='dense', summary=False, model="llama3.3", chunks_only=False):
    response = client.embeddings(prompt=query, model="mxbai-embed-large")
    query_embedding = response["embedding"]

    top_k_docs = faiss_retriever._retrieve(query_embedding, top_k=top_k)

    bm25_retriever = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=top_k,
    stemmer=Stemmer.Stemmer("english"),
    language="english",
    )
    retrieved_nodes = bm25_retriever.retrieve(query)

    results = {'faiss': top_k_docs, 'bm25':retrieved_nodes}
    ranked_results = hybrid_embedding(results, top_k=top_k)

    if mode == 'dense':
        print('using FAISS')
        ans_nodes =top_k_docs
    elif mode == 'sparse':
        print('using BM25')
        ans_nodes = retrieved_nodes
    else:
        print('using Hybrid')
        ans_nodes = ranked_results

    parent_flag = True
    context = set([get_document_by_chunk_metadata(docs).text for docs in ans_nodes])
    if chunks_only:
        parent_flag = False
        print('using chunks only')
        context = [docs.node.text for docs in ans_nodes]
        
    if summary:
        print('using summaries')
        context_nodes = remove_duplicate_documents([get_document_by_chunk_metadata(docs) for docs in ans_nodes])

        if chunks_only:
            context_nodes=ans_nodes
        summaries = summarize_each_chunk(context_nodes, client, model='llama3.3', query=query,parent=parent_flag)
        context = summaries

    answer = generate_response_with_cot(context, query, choices, client, model=model)

    # Format the references
    references = []
    for i, doc in enumerate(ranked_results[:top_k], start=1):
        metadata = doc.metadata
        source_info = f"Source {i}: {metadata['title']} (Page {metadata['page']}, Folder: {metadata['folder']})"
        references.append(source_info)

    return answer, "\n".join(references), context

# Evaluation

In [23]:
# Generate prompts dynamically
def generate_choices(row):
    options = []
    for choice in ['A', 'B', 'C', 'D', 'E']:
        # Check for NaN or blank values
        if pd.notna(row[choice]) and row[choice] != '':
            options.append(f"{choice}. {row[choice]}")
    
    # Construct the prompt with few-shot examples
    choices = "\n".join(options)
    
    return choices

In [24]:
from sklearn.model_selection import train_test_split
# Load the Excel file
file_path = '/home/jeryl4913/lto_rag_reviewer/notebooks/eval/LTO_EXAM.csv'
df = pd.read_csv(file_path)

# Display the first few rows of each set
print("Testing Data:")
print(len(df))
display(df.head())

Testing Data:
60


,Question,A,B,C,D,E,Answer,Choices
0,"If the driver is turning left, he must: (U-tur...",have the right of way,do so slowly with caution,yield to approaching cars,NaN,NaN,C,A. have the right of way\nB. do so slowly with...
1,"When a vehicle starts to skid, what should the...",Immediately step on the brakes,Hold firmly on to the wheel while slowing down...,Turn the wheels tp the opposite the direction ...,NaN,NaN,B,A. Immediately step on the brakes\nB. Hold fir...
2,"If you are parking uphill without a curb, turn...",edge of the street,other side of the street,middle of the street,NaN,NaN,A,A. edge of the street\nB. other side of the st...
3,"When parking downhill, you should turn your fr...",toward the curb of the sidewalk,away from the curb,any direction will do,NaN,NaN,A,A. toward the curb of the sidewalk\nB. away fr...
4,Which of the following is the maximum speed li...,60 kph,80 kph,100 kph,NaN,NaN,C,A. 60 kph\nB. 80 kph\nC. 100 kph


In [25]:
import time

df["AI"] = np.nan
ai_answer = []
context_answer = []
inference_times = []
for i in tqdm(range(len(df))):
    start_time = time.time()
    answ = gen_query(df["Question"].iloc[i], df["Choices"].iloc[i], top_k=15, client=client, mode='dense', model="llama3.3")
    ai_answer.append(answ[0])
    context_answer.append(answ[2])
    end_time = time.time()
    inference_times.append(end_time - start_time)

df["AI"] = ai_answer
df["Inference_times"] = inference_times
df["Context"] = context_answer
print("Updated Testing Data with AI Answers:")
display(df.head())

  0%|          | 0/60 [00:00<?, ?it/s]

using FAISS
Context: If the driver is turning left, he must: (U-turn / Left Rule)...
Summary: Drivers turning left must follow the 'Left Rule', which may be an unmarked or marked lane requirement.
------------------------------------------------------------


  2%|▏         | 1/60 [00:31<30:31, 31.05s/it]

using FAISS
Context: When a vehicle starts to skid, what should the driver do?...
Summary: Drivers should take immediate action by gently steering into the direction of the skid while taking their foot off the gas pedal. This helps correct the vehicle's trajectory and regain traction.
------------------------------------------------------------


  3%|▎         | 2/60 [01:04<31:06, 32.18s/it]

...

 98%|█████████▊| 59/60 [34:55<00:36, 36.05s/it]

using FAISS
Context: You may never park:...
Summary: Many cities are adopting 
------------------------------------------------------------


100%|██████████| 60/60 [35:34<00:00, 35.57s/it]

Updated Testing Data with AI Answers:


,Question,A,B,C,D,E,Answer,Choices,AI,Inference_times,Context
0,"If the driver is turning left, he must: (U-tur...",have the right of way,do so slowly with caution,yield to approaching cars,NaN,NaN,C,A. have the right of way\nB. do so slowly with...,[Left Rule],31.046930,{I I A. GRAVE VIOLATIONS ( 5 Demerit Points) ...
1,"When a vehicle starts to skid, what should the...",Immediately step on the brakes,Hold firmly on to the wheel while slowing down...,Turn the wheels tp the opposite the direction ...,NaN,NaN,B,A. Immediately step on the brakes\nB. Hold fir...,[],32.962109,{control of their vehicles.Anticipate Slippery...
2,"If you are parking uphill without a curb, turn...",edge of the street,other side of the street,middle of the street,NaN,NaN,A,A. edge of the street\nB. other side of the st...,[A],31.688846,{How to Handle Pavement Drop-Off While Driving...
3,"When parking downhill, you should turn your fr...",toward the curb of the sidewalk,away from the curb,any direction will do,NaN,NaN,A,A. toward the curb of the sidewalk\nB. away fr...,[A],36.234869,{How to Handle Pavement Drop-Off While Driving...
4,Which of the following is the maximum speed li...,60 kph,80 kph,100 kph,NaN,NaN,C,A. 60 kph\nB. 80 kph\nC. 100 kph,[None of the above],37.051482,{cash/e-cardseasily accessible to minimize del...


In [26]:
import re


def process_answers(answers):
    formatted_answers = []
    
    for a in answers:
        
        matches = re.findall(r'\[?\s*([A-E](?:\s*,\s*[A-E])*)\s*\]?', str(a)) # Extract answers like [A, C, D] or [A] or [B, D]
        answers = []
        for match in matches:
            answers.extend(re.split(r'\s*,\s*', match))  # Split by comma and remove spaces
        unique_sorted_answers = sorted(set(answers), key=lambda x: ['A', 'B', 'C', 'D', 'E'].index(x))
        if not unique_sorted_answers:
            formatted_answers.append(None)
        else:
            formatted_answers.append(unique_sorted_answers)
    return formatted_answers

df_results = df[["Question", "Answer", "AI", "Inference_times"]].copy()
df_results['Answer'] = df_results['Answer'].apply(lambda x: x.split(', '))
df_results['AI'] = process_answers(df_results["AI"])
df_results['Answer'] = process_answers(df_results["Answer"])



def calculate_scores(df):
    scores = []
    for index, row in df.iterrows():
        correct_answers = set(row['Answer'] if row['Answer'] is not None else [])
        ai_answers = set(row['AI'] if row['AI'] is not None else [])
        if ai_answers == correct_answers:
            score = 1.0
        else:
            score = 0.0
        scores.append(score)
    
    df['Score'] = scores
    accuracy = scores.count(1.0) / len(scores)
    print(f'Final Score: {scores.count(1.0):.2f}/{len(scores):.2f}')
    print(f'Accuracy: {accuracy:.2f}%')
    return df

# Apply the scoring function
scored_df = calculate_scores(df_results)

# Display the dataframe to verify the results
display(scored_df[['Question', 'Answer', 'AI', 'Score', 'Inference_times']])

Final Score: 36.00/60.00
Accuracy: 0.60%


,Question,Answer,AI,Score,Inference_times
0,"If the driver is turning left, he must: (U-tur...",[C],None,0.0,31.046930
1,"When a vehicle starts to skid, what should the...",[B],None,0.0,32.962109
2,"If you are parking uphill without a curb, turn...",[A],[A],1.0,31.688846
3,"When parking downhill, you should turn your fr...",[A],[A],1.0,36.234869
4,Which of the following is the maximum speed li...,[C],None,0.0,37.051482
5,Never park or stop at the side of the road wit...,[C],None,0.0,31.251813
6,While driving with maximum speed and you have ...,[C],[C],1.0,37.153632
7,What will happen when your rear tire blows out?,[B],[A],0.0,36.816878
8,Operating a public utility vehicle equipped wi...,[A],"[A, B, C]",0.0,36.154871
9,Your speed while driving at night should keep on:,[A],"[A, C]",0.0,33.217110


In [27]:
# Generate prompts dynamically
def generate_prompt(row):
    
    # Construct the prompt with few-shot examples
    prompt = f"\nActual Question: {row['Question']}\n" 
    prompt += "\nPlease answer the question based on the given context."
    
    return prompt

In [28]:
# Load the Excel file
file_path = '/home/jeryl4913/lto_rag_reviewer/notebooks/eval/LTO_EXAM_QnA.csv'
df = pd.read_csv(file_path, encoding='ISO-8859-1')
df['Prompt'] = df.apply(generate_prompt, axis=1)
display(df.head())

,Question,Answer,Prompt
0,Traffic Jam can be prevented if you,Keep opposing lanes open,\nActual Question: Traffic Jam can be prevente...
1,When making a right turn you should,Stay on the outermost lane of the road then si...,\nActual Question: When making a right turn yo...
2,"When you intend to turn right or left, signal ...",25 meters before you intend to make your turn,\nActual Question: When you intend to turn rig...
3,"At an intersection with a traffic light, make ...",The green light is on and there is a left turn...,\nActual Question: At an intersection with a t...
4,Graft and corruption in the traffic enforcemen...,Self disciplined by drivers and obeying traffi...,\nActual Question: Graft and corruption in the...


In [29]:
def generate_response_with_notice(summaries, query, client, model="llama3.3"):
    # Combine summaries into context block
    context = "\n".join(summaries)
    
    # Create prompt to answer based on summarized text
    prompt = f"""
    Use the following summarized information to answer the query accurately and concisely. 
    DO NOT USE BACKGROUND KNOWLEDGE OUTSIDE THE CONTEXT PROVIDED.
    If the information is not sufficient to fully address the query, respond ONLY with:
    "The available information is insufficient to provide a complete answer to this query."

    Summarized Context:
    {context}
    
    Query:
    {query}
    
    Response:
    """
    
    # Send the prompt to Ollama
    response = client.generate(
        model=model,
        prompt=prompt
    )
    
    return response['response'].strip()

In [33]:

def summarize_context(context: str, client, model: str) -> str:
    
    prompt_instruction = '''
    You are an expert at distilling information into concise summaries. 
    Read the following context carefully and provide a clear, concise, and accurate summary of the key points. 
    Ensure that the summary captures the essence of the content without omitting critical details. 
    
    Provide the summarized output enclosed in double quotes.
    '''

    few_shot_examples = '''
    Here is an example of how to do this

    Example 1:
    Context: High-dimensional feature vectors, derived from complex embedding models, are now critical for applications like e-commerce product search and recommendation, document retrieval, facial recognition, and retrieval-based large language models (LLMs), Pan, J. J., et al.(2024). Traditional database management systems, however, struggle with the unique characteristics of feature vectors due to the nuance of semantic similarity, expensive computational cost, large data size, and lack of indexable structure. Numerous vector database management systems (VDBMSs) have been created to tackle these issues. Typical VDBMSs consist of a query processor that handles different query types and a storage manager that manages large-scale vector indexing, storage, and compression. One key aspect of query processing is similarity scores which is important in capturing similarity relationships between feature vectors. There are various different vector indexing structures and techniques for efficient vector search. Common structures include table-based, tree-based, and graph-based.  Table-based indexes partition vectors into buckets with techniques including locality sensitive hashing (LSH), learning to hash (L2H), and quantization for compression.
    Output: "High-dimensional feature vectors are essential for applications like e-commerce search, document retrieval, facial recognition, and retrieval-based LLMs. Traditional databases struggle to manage these vectors due to their size, computational cost, and the complexity of semantic similarity. Vector database management systems (VDBMSs) address these challenges with specialized query processors and storage managers for indexing, storage, and compression. Efficient vector search relies on similarity scores and various indexing techniques, such as table-based, tree-based, and graph-based structures, with methods like locality sensitive hashing (LSH) and quantization playing a key role."
    
    Now, summarize the following context and provide the response inside double quotes:
    '''
    
    full_prompt = prompt_instruction + few_shot_examples + f"\nContext: {context}"
    
    try:
        response = client.generate(
            model=model,
            prompt=full_prompt
        )

        # Extract text within double quotes (literal summary)
        matches = re.findall(r'"([^"]*)"', response['response'].strip())

        print(f"Context: {context[:100]}...")  # Print first 100 chars for readability
        print(f"Summary: {matches[0] if matches else 'No match found'}")
        print('-' * 60)

        if matches:
            return matches[0]
        else:
            return "Summary not available."
        
    except Exception as e:
        print(f"Error during summarization: {e}")
        return "Error encountered during summarization."

def cot_qa_format(query, context, client, model):
    return f"""

    Let's break down the problem step by step:
    1. Identify the key components of the following question: {query}
    2. Decompose the retrieved context into key components and extract relevant information step by step. Here is the context: {summarize_context(query, client, model)}
    3. Provide the best possible answer based on the reasoning process.
    """

def generate_response_with_cot(summaries, query, client, model="llama3.1:8b", cot_model="llama3.1:8b", para_model="llama3.1:8b"):
    """
    Generate a response using Chain of Thought (CoT) reasoning by passing intermediate
    results through successive steps to build a synthesized final answer.
    """

    # 1. Combine summaries into context block
    context = "\n".join(summaries)

    try:
        # 2. Step 1: Generate initial thought process (CoT step 1)
        cot_query = cot_qa_format(query, context, client, para_model)
        response1 = client.generate(model=cot_model, prompt=cot_query)
        reasoning1 = response1['response'].strip()

        # 3. Step 2: Refine based on first result
        cot_query2 = f"""
        We reasoned that: {reasoning1}
        Can you expand on this by addressing additional details or gaps?
        {query}
        """
        response2 = client.generate(model=cot_model, prompt=cot_query2)
        reasoning2 = response2['response'].strip()

        # 5. Aggregate all steps for the final decision
        final_query = f"""
        Based on the results:
        - Step 1: {reasoning1}
        - Step 2: {reasoning2}

        Now, synthesize these results and answer this question:
        {query + " Answer which of the choices match the synthesized results."}

        """
        
        # 6. Final model call to produce the overall response
        final_response = client.generate(model=model, prompt=final_query)
        
        return final_response['response'].strip()


    except Exception as e:
        print(f"Error processing prompt: {e}")
        return None



In [34]:
def gen_query(query, top_k, client, mode='dense', summary=False, model="llama3.3", chunks_only=False):
    response = client.embeddings(prompt=query, model="mxbai-embed-large")
    query_embedding = response["embedding"]

    top_k_docs = faiss_retriever._retrieve(query_embedding, top_k=top_k)

    bm25_retriever = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=top_k,
    stemmer=Stemmer.Stemmer("english"),
    language="english",
    )
    retrieved_nodes = bm25_retriever.retrieve(query)

    results = {'faiss': top_k_docs, 'bm25':retrieved_nodes}
    ranked_results = hybrid_embedding(results, top_k=top_k)

    if mode == 'dense':
        print('using FAISS')
        ans_nodes =top_k_docs
    elif mode == 'sparse':
        print('using BM25')
        ans_nodes = retrieved_nodes
    else:
        print('using Hybrid')
        ans_nodes = ranked_results

    parent_flag = True
    context = set([get_document_by_chunk_metadata(docs).text for docs in ans_nodes])
    if chunks_only:
        parent_flag = False
        print('using chunks only')
        context = [docs.node.text for docs in ans_nodes]
        
    if summary:
        print('using summaries')
        context_nodes = remove_duplicate_documents([get_document_by_chunk_metadata(docs) for docs in ans_nodes])

        if chunks_only:
            context_nodes=ans_nodes
        summaries = summarize_each_chunk(context_nodes, client, model='llama3.3', query=query,parent=parent_flag)
        context = summaries

    answer = generate_response_with_cot(context, query, client, model=model)

    # Format the references
    references = []
    for i, doc in enumerate(ranked_results[:top_k], start=1):
        metadata = doc.metadata
        source_info = f"Source {i}: {metadata['title']} (Page {metadata['page']}, Folder: {metadata['folder']})"
        references.append(source_info)

    return answer, "\n".join(references), context

In [35]:
import time
qr_range = (0,60)
df["AI"] = np.nan
ai_answer = []
inference_times = []

for i in tqdm(range(*qr_range)):
    start_time = time.time()
    ai_answer.append(gen_query(df.loc[i,"Prompt"],  top_k=15, client=client, mode='dense', model="llama3.3"))
    end_time = time.time()
    inference_times.append(end_time - start_time)

  0%|          | 0/60 [00:00<?, ?it/s]

using FAISS
Context: 
Actual Question: Traffic Jam can be prevented if you

Please answer the question based on the given...
Summary: Traffic jams can be prevented or minimized by adopting alternative modes of transportation, optimizing traffic flow through intelligent systems, reducing congestion by avoiding peak hours, and implementing smart infrastructure such as intelligent intersections.
------------------------------------------------------------


  2%|▏         | 1/60 [00:56<56:01, 56.97s/it]

using FAISS
Context: 
Actual Question: When making a right turn you should

Please answer the question based on the given...
Summary: Before executing a right turn, it's essential to check for oncoming traffic and pedestrians. The correct procedure typically involves yielding to other drivers and pedestrians in the adjacent lane, signaling your intention to turn, and checking that the way ahead is clear before proceeding with the turn.
------------------------------------------------------------


  3%|▎         | 2/60 [01:50<53:20, 55.18s/it]

...

 97%|█████████▋| 58/60 [41:24<01:20, 40.33s/it]

using FAISS
Context: 
Actual Question: The mark for a railway crossing

Please answer the question based on the given con...
Summary: No match found
------------------------------------------------------------


 98%|█████████▊| 59/60 [42:16<00:43, 43.58s/it]

using FAISS
Context: 
Actual Question: Children must be held in approved child restraint if there are

Please answer the ...
Summary: Under 40 pounds (or approximately 18 kg) and/or under 4 years old.
------------------------------------------------------------


100%|██████████| 60/60 [42:50<00:00, 42.84s/it]


In [36]:
df.loc[qr_range[0]:qr_range[1]-1, "AI"] = [answ[0] for answ in ai_answer]
df.loc[qr_range[0]:qr_range[1]-1, "Context"] = [" ".join(map(str, answ[2])) for answ in ai_answer]
df.loc[qr_range[0]:qr_range[1]-1, "Inference_Time"] = inference_times 
df_new = df.loc[qr_range[0]:qr_range[1]-1].copy()
df = df_new.copy()

/tmp/ipykernel_37600/3496959586.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['Based on the synthesized results, the best possible answer to the question "Traffic Jam can be prevented if you" is:\n\n**Adopt alternative modes of transportation and optimize traffic flow through intelligent systems.**\n\nThis answer encompasses the key strategies identified in the context, including:\n\n1. Promoting alternative modes of transportation (e.g., public transport, carpooling, cycling)\n2. Optimizing traffic flow through intelligent systems (e.g., smart traffic lights, monitoring systems)\n\nAdditionally, other relevant answers that match the synthesized results include:\n\n* **Avoid peak hours to reduce congestion**\n* **Implement smart infrastructure such as intelligent intersections**\n\nThese options are all supported by the context and can help prevent traffic jams.\n\nSo, the matching choices are:\n\

# 10. Similarity Evaluation

In [37]:
from llama_index.core.evaluation import SemanticSimilarityEvaluator
from llama_index.core.base.embeddings.base import BaseEmbedding
import asyncio
from llama_index.core.embeddings import resolve_embed_model
from pydantic import PrivateAttr

class OllamaEmbeddingModel(BaseEmbedding):
    _client: Client = PrivateAttr()

    def __init__(self, model_name: str = "mxbai-embed-large", timeout: int = 300):
        super().__init__()
        self.model_name = model_name
        self._client = Client() 

    async def _aget_query_embedding(self, query: str) -> list[float]:
        return await self._aget_text_embedding(query)

    async def _aget_text_embedding(self, text: str) -> list[float]:
        loop = asyncio.get_event_loop()
        embedding_response = await loop.run_in_executor(
            None, self._client.embeddings, self.model_name, text
        )
        return embedding_response['embedding']  

    def _get_query_embedding(self, query: str) -> list[float]:
        return self._get_text_embedding(query)

    def _get_text_embedding(self, text: str) -> list[float]:
        embedding_response = self._client.embeddings(
            model=self.model_name,
            prompt=text
        )
        return embedding_response['embedding']


embed_model = OllamaEmbeddingModel(model_name="mxbai-embed-large")
evaluator = SemanticSimilarityEvaluator(
    embed_model=embed_model,
    similarity_threshold=0.6
)

results_scores = []
results_passing = []
for i in tqdm(range(len(df))):
    response = df.loc[i, "AI"]
    reference = df.loc[i, "Answer"]

    result = await evaluator.aevaluate(
    response=response,
    reference=reference,
    )
    results_scores.append(result.score)
    results_passing.append(result.passing)
    
df['Score'] = results_scores
df['Passing'] = results_passing

average_score = df['Score'].mean()
total_items = len(df)
passing_items = df['Passing'].sum()  
print(f"Average Score: {average_score:.4f}")
print(f"Passing: {passing_items}/{total_items}") 
display(df[['Question', 'Answer', 'AI', 'Score', 'Passing', 'Inference_Time']])

100%|██████████| 60/60 [00:04<00:00, 13.75it/s]

Average Score: 0.6348
Passing: 43/60


,Question,Answer,AI,Score,Passing,Inference_Time
0,Traffic Jam can be prevented if you,Keep opposing lanes open,"Based on the synthesized results, the best pos...",0.624936,True,56.972070
1,When making a right turn you should,Stay on the outermost lane of the road then si...,"Based on the provided context, when making a r...",0.768676,True,53.918721
2,"When you intend to turn right or left, signal ...",25 meters before you intend to make your turn,"When you intend to turn right or left, signal ...",0.772739,True,33.576814
3,"At an intersection with a traffic light, make ...",The green light is on and there is a left turn...,"Based on the synthesized results, the correct ...",0.778546,True,46.873964
4,Graft and corruption in the traffic enforcemen...,Self disciplined by drivers and obeying traffi...,Based on the provided breakdown and expanded e...,0.619826,True,47.836722
5,"On a four(4) lane road with single white line,...",Overtake by passing over the solid white line,Based on the provided analysis and general kno...,0.769908,True,71.307832
6,A double solid yellow line with broken white l...,Absolutely no overtaking,"Based on the synthesized results, a double sol...",0.535426,False,46.118041
7,"When making a U-Turn, you should",Check for traffic behind you and indicate your...,"When making a U-Turn, you should **signal your...",0.728841,True,38.615253
8,Signs that are triangular in shape and with a ...,Caution or warning signs,Based on the provided context and the step-by-...,0.749514,True,42.734952
9,"Signs that are round, inverted triangle or oct...",Regulatory signs,Based on the provided context and the step-by-...,0.658674,True,33.604557


# 11. Relevancy Evaluation

In [39]:
from llama_index.core.evaluation import RelevancyEvaluator
from llama_index.llms.ollama import Ollama

ollama_llm = Ollama(model="llama3.1:8b", request_timeout=300)
evaluator = RelevancyEvaluator(llm=ollama_llm)

eval_results = []

for i in tqdm(range(len(df))):
    eval_result = await evaluator.aevaluate(
        query=df.loc[i, "Question"],
        response=df.loc[i, "AI"],
        contexts=[df.loc[i, "Context"]]  
    )
    eval_results.append(eval_result.passing)

df['Eval'] = eval_results

total_items = len(df)
passing_items = df['Eval'].sum()
score = f"Score: {passing_items}/{total_items}"
percentage = passing_items / total_items if total_items > 0 else 0
print(score)
print(f"Percentage: {percentage:.2%}")
display(df[['Question', 'Answer', 'AI', 'Eval', 'Inference_Time']])

100%|██████████| 60/60 [08:12<00:00,  8.21s/it]

Score: 52/60
Percentage: 86.67%


,Question,Answer,AI,Eval,Inference_Time
0,Traffic Jam can be prevented if you,Keep opposing lanes open,"Based on the synthesized results, the best pos...",True,56.972070
1,When making a right turn you should,Stay on the outermost lane of the road then si...,"Based on the provided context, when making a r...",True,53.918721
2,"When you intend to turn right or left, signal ...",25 meters before you intend to make your turn,"When you intend to turn right or left, signal ...",True,33.576814
3,"At an intersection with a traffic light, make ...",The green light is on and there is a left turn...,"Based on the synthesized results, the correct ...",True,46.873964
4,Graft and corruption in the traffic enforcemen...,Self disciplined by drivers and obeying traffi...,Based on the provided breakdown and expanded e...,True,47.836722
5,"On a four(4) lane road with single white line,...",Overtake by passing over the solid white line,Based on the provided analysis and general kno...,True,71.307832
6,A double solid yellow line with broken white l...,Absolutely no overtaking,"Based on the synthesized results, a double sol...",True,46.118041
7,"When making a U-Turn, you should",Check for traffic behind you and indicate your...,"When making a U-Turn, you should **signal your...",True,38.615253
8,Signs that are triangular in shape and with a ...,Caution or warning signs,Based on the provided context and the step-by-...,True,42.734952
9,"Signs that are round, inverted triangle or oct...",Regulatory signs,Based on the provided context and the step-by-...,True,33.604557
